In [24]:
from tokenize import Number
import pyspark
import pyspark.sql.functions
from pyspark.sql.functions import round
from pyspark.sql.functions import regexp_replace
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from pyspark.sql.functions import array_union
from pyspark.sql.functions import array
from pyspark.sql.functions import size
from pyspark.sql.functions import when


spark = SparkSession.builder.master("local[1]") \
                    .appName('SparkLab2') \
                    .getOrCreate()

tiktokData2019 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/home/student/Desktop/TikTok_songs_2019.csv")
tiktokData2020 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/home/student/Desktop/TikTok_songs_2020.csv")
tiktokData2021 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/home/student/Desktop/TikTok_songs_2021.csv")
tiktokData2022 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/home/student/Desktop/TikTok_songs_2022.csv")

In [115]:
from pyspark.sql.functions import input_file_name
from pyspark.sql.functions import regexp_extract
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import lit

df19 = tiktokData2019\
.select('artist_pop','track_name', 'artist_name', 'danceability', 'album')\
.withColumn('year', lit(2019))

df20 = tiktokData2020\
.select('artist_pop','track_name', 'artist_name', 'danceability', 'album')\
.withColumn('year', lit(2020))

df21 = tiktokData2021\
.select('artist_pop','track_name', 'artist_name', 'danceability', 'album')\
.withColumn('year', lit(2021))

df22 = tiktokData2022\
.select('artist_pop','track_name', 'artist_name', 'danceability', 'album')\
.withColumn('year', lit(2022))

df = df19.unionByName(df20).unionByName(df21).unionByName(df22)

df.select('artist_pop','track_name', 'artist_name', 'danceability', 'album')

tiktokData = df.sort(desc("danceability")).select("track_name", "artist_name", "year", "danceability").show(6)


+--------------------+-----------+----+------------+
|          track_name|artist_name|year|danceability|
+--------------------+-----------+----+------------+
|  Dancing in My Room|   347aidan|2020|        0.98|
|                 Uno|    Ambjaay|2019|       0.978|
|I'm Looking for M...|   Memetown|2020|       0.977|
|           Hood Baby|       KBFR|2020|       0.974|
|              MICKEY| Lil Yachty|2019|       0.971|
|Muffins In The Fr...|      Tiagz|2020|       0.965|
+--------------------+-----------+----+------------+
only showing top 6 rows



In [130]:
from pyspark.sql.functions import col
from pyspark.sql.functions import collect_list
from pyspark.sql.functions import count

df1 = df\

df1 = df.select('artist_name',"artist_pop",'track_name', 'album')\
.groupBy('artist_name','track_name', 'album')\
.agg(collect_list("artist_pop").alias('popularity'))\
.withColumn("other", size("popularity"))\
.where("other > 1")\


df1.show()
# df2 = df1.select('artist_name',"popularity")\

# df2.show(1000)

+--------------+--------------------+--------------------+------------+-----+
|   artist_name|          track_name|               album|  popularity|other|
+--------------+--------------------+--------------------+------------+-----+
|      24kGoldn|Mood (feat. iann ...|Mood (feat. iann ...|    [74, 74]|    2|
|      24kGoldn|           VALENTINO|DROPPED OUTTA COL...|    [74, 74]|    2|
|     Absofacto|            Dissolve|     Thousand Peaces|    [56, 56]|    2|
|       Amaarae|SAD GIRLZ LUV MON...|SAD GIRLZ LUV MON...|    [64, 64]|    2|
|  Ant Saunders|       Yellow Hearts|       Yellow Hearts|    [52, 52]|    2|
| Ariana Grande|               34+35|           Positions|    [87, 87]|    2|
|Arizona Zervas|             ROXANNE|             ROXANNE|[66, 66, 66]|    3|
|   Audrey Mika|Y U Gotta B Like ...|Y U Gotta B Like ...|    [56, 56]|    2|
|   Bag Raiders|      Shooting Stars|Bag Raiders (Deluxe)|    [53, 53]|    2|
|  Bella Poarch|       Build a Bitch|       Build a Bitch|    [6

In [138]:
from pyspark.sql.functions import array_max
from pyspark.sql.functions import when

df2 = df1\

df2 = df1.select('track_name','artist_name', 'album' , array_max("popularity").alias('artist_pop'))

df2.sort(desc('artist_pop')).show(1000)

# 'track_name','album'

+--------------------+--------------------+--------------------+----------+
|          track_name|         artist_name|               album|artist_pop|
+--------------------+--------------------+--------------------+----------+
|     Save Your Tears|          The Weeknd|         After Hours|        93|
|     Blinding Lights|          The Weeknd|         After Hours|        93|
|          Bad Habits|          Ed Sheeran|          Bad Habits|        90|
|I Don't Care (wit...|          Ed Sheeran|I Don't Care (wit...|        90|
|Peaches (feat. Da...|       Justin Bieber|             Justice|        90|
|      Break My Heart|            Dua Lipa|    Future Nostalgia|        88|
|     Don't Start Now|            Dua Lipa|     Don't Start Now|        88|
|               Freak|            Doja Cat|               Freak|        88|
|Like That (feat. ...|            Doja Cat|            Hot Pink|        88|
|          Boss Bitch|            Doja Cat|          Boss Bitch|        88|
|        Nee